# Demo and Comparison of Big Data File Formats

### CSV and JSON
Old data formats that are not designed for big data and scaling  
**Typical feature:** human-readable

### Avro, ORC, Parquet
First generation of special big data formats that allow fast writes, fast reads or both  
**Typical features:** splittable, compressible, data skipping and predicat pushdown, data schema included

### Delta, Iceberg, Hudi
Latest generation of big data formats that support ACID transactions, audit save transaction logs and time travel  
**Typical features:** enhancing first generation format with additonal meta data and read/write procedures

## 1. Import

#### Import the necessary libraries for data processing using PySpark. Some of the important imports include:
- SparkContext and SparkConf from pyspark: these libraries are used to initialize the Spark cluster and set up the configuration for the cluster.
- SparkSession and SQLContext from pyspark.sql: These libraries are used for creating and interacting with Spark SQL contexts.


In [79]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark.sql.functions import col

from delta import *

import datetime
from datetime import datetime, timedelta

import json
import csv

# use 95% of the screen for jupyter cell
from IPython.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

In [56]:
spark


## 2. Launch Spark Jupyter and Configuration

#### Configure a Spark session for Kubernetes cluster with S3 support
### CLUSTER MANAGER
- set the Kubernetes master URL as Cluster Manager(“k8s://https://” is NOT a typo, this is how Spark knows the “provider” type)

### KUBERNETES
- set the namespace that will be used for running the driver and executor pods
- set the docker image from which the Worker/Exectutor pods are created
- set the Kubernetes service account name and provide the authentication details for the service account (required to create worker pods)

### SPARK
- set the driver host and the driver port (find name of the driver service with 'kubectl get services' or in the helm chart configuration)
- enable Delta Lake, Iceberg, and Hudi support by setting the spark.sql.extensions
- configure Hive catalog for Iceberg
- enable S3 connector
- set the number of worker pods, their memory and cores (HINT: number of possible tasks = cores * executores)

### SPARK SESSION
- create the Spark session using the SparkSession.builder object
- get the Spark context from the created session and set the log level to "ERROR".


In [18]:
spark.stop()

In [2]:
appName="jupyter-spark"

conf = SparkConf()

# CLUSTER MANAGER

conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local:443")

# CONFIGURE KUBERNETES

conf.set("spark.kubernetes.namespace","frontend")
conf.set("spark.kubernetes.container.image", "thinkportgmbh/workshops:spark-3.3.1")
conf.set("spark.kubernetes.container.image.pullPolicy", "Always")

conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
conf.set("spark.kubernetes.authenticate.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")
conf.set("spark.kubernetes.authenticate.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token")

# CONFIGURE SPARK

conf.set("spark.sql.session.timeZone", "Europe/Berlin")
conf.set("spark.driver.host", "jupyter-spark-driver.frontend.svc.cluster.local")
conf.set("spark.driver.port", "29413")

conf.set("spark.jars", "/opt/spark/jars/spark-avro_2.12-3.3.1.jar")
conf.set("spark.driver.extraClassPath","/opt/spark/jars/spark-avro_2.12-3.3.1.jar")
conf.set("spark.executor.extraClassPath","/opt/spark/jars/spark-avro_2.12-3.3.1.jar")

conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions, org.apache.spark.sql.hudi.HoodieSparkSessionExtension")

######## Hive als Metastore einbinden
conf.set("hive.metastore.uris", "thrift://hive-metastore.hive.svc.cluster.local:9083") 

######## Iceberg configs
conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
conf.set("spark.sql.catalog.ice","org.apache.iceberg.spark.SparkCatalog") 
conf.set("spark.sql.catalog.ice.type","hive") 
conf.set("spark.sql.catalog.ice.uri","thrift://hive-metastore.hive.svc.cluster.local:9083") 


####### Hudi configs
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

# CONFIGURE S3 CONNECTOR
conf.set("spark.hadoop.fs.s3a.endpoint", "minio.minio.svc.cluster.local:9000")
conf.set("spark.hadoop.fs.s3a.access.key", "trainadm")
conf.set("spark.hadoop.fs.s3a.secret.key", "train@thinkport")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

# CONFIGURE WORKER (Customize based on workload)

conf.set("spark.executor.instances", "1")
conf.set("spark.executor.memory", "1G")
conf.set("spark.executor.cores", "2")

# SPARK SESSION

spark = SparkSession\
    .builder\
    .config(conf=conf) \
    .config('spark.sql.session.timeZone', 'Europe/Berlin') \
    .appName(appName)\
    .enableHiveSupport() \
    .getOrCreate()


sc=spark.sparkContext
sc.setLogLevel("ERROR")

# get the configuration object to check all the configurations the session was startet with
for entry in sc.getConf().getAll():
        if entry[0] in ["spark.app.name","spark.kubernetes.namespace","spark.executor.memory","spark.executor.cores","spark.driver.host","spark.master","spark.sql.extensions"]:
            print(entry[0],"=",entry[1])

23/05/14 20:09:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


spark.kubernetes.namespace = frontend
spark.sql.extensions = io.delta.sql.DeltaSparkSessionExtension, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions, org.apache.spark.sql.hudi.HoodieSparkSessionExtension
spark.master = k8s://https://kubernetes.default.svc.cluster.local:443
spark.app.name = jupyter-spark
spark.executor.memory = 1G
spark.executor.cores = 2
spark.driver.host = jupyter-spark-driver.frontend.svc.cluster.local


## Create sample data

In [3]:
# initial daten
account_data1 = [
    (1,"alex","2019-01-01",1000),
    (2,"alex","2019-02-01",1500),
    (3,"alex","2019-03-01",1700),
    (4,"maria","2020-01-01",5000)
    ]

# update mit Änderung und neuem Datensat
account_data2 = [
    (1,"alex","2019-03-01",3300),
    (2,"peter","2021-01-01",100)
    ]

# Update mit neuer Spalte
account_data3 = [
    (1,"otto","2019-10-01",4444,"neue Spalte 1")
]

schema = ["id","account","dt_transaction","balance"]
schema3 = ["id","account","dt_transaction","balance","new"]

df1 = spark.createDataFrame(data=account_data1, schema = schema).withColumn("dt_transaction",col("dt_transaction").cast("date")).repartition(3)
df2 = spark.createDataFrame(data=account_data2, schema = schema).withColumn("dt_transaction",col("dt_transaction").cast("date")).repartition(3)
df3 = spark.createDataFrame(data=account_data3, schema = schema3).withColumn("dt_transaction",col("dt_transaction").cast("date")).repartition(3)

print("++ create new dataframe and show schema and data")
print("################################################")

# df1.printSchema()
# df1.show(truncate=False)
# df3.show(truncate=False)

++ create new dataframe and show schema and data
################################################


## Configure boto3

In [11]:
import boto3
from botocore.client import Config

options = {
    'endpoint_url': 'http://minio.minio.svc.cluster.local:9000',
    'aws_access_key_id': 'trainadm',
    'aws_secret_access_key': 'train@thinkport',
    'config': Config(signature_version='s3v4'),
    'verify': False}

s3_resource = boto3.resource('s3', **options)  

s3_client = boto3.client('s3', **options)

bucket = "fileformats"
bucket_path="s3://"+bucket

In [9]:
def ls(bucket,prefix):
    '''List objects from bucket/prefix'''
    try:
        for obj in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix):
            print(obj.key)
    except Exception as e: 
        print(e)
    
    
def cat(bucket,prefix,binary=False):
    '''Show content of one or several files with same prefix/wildcard'''
    try:
        for obj in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix):
            print("File:",obj.key)
            print("----------------------")
            if binary==True:
                print(obj.get()['Body'].read())
            else: 
                print(obj.get()['Body'].read().decode())
            print("######################")
    except Exception as e: 
        print(e)
            
def rm(bucket,prefix):
    '''Delete everything from bucket/prefix'''
    for object in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix):
        print(object.key)
        s3_client.delete_object(Bucket=bucket, Key=object.key)
    print(f"Deleted files from {bucket}/{prefix}*")


In [14]:
# show everything in bucket
ls(bucket,"")
print("#############################")
# show folder
ls(bucket,"csv")
print("#############################")
# show subfolder
ls(bucket,"delta/_delta_log/")
print("#############################")
print("")
# show content of one or several files with same prefix/wildcard
cat(bucket,'csv/part')

#############################
#############################
#############################



In [ ]:
#rm("aleks-test", "iceberg")

## CSV

In [15]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_csv=(df1
           .write
           .format("csv")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/csv")
          )


Number of Partitions: 3


In [16]:
ls(bucket,"csv")

csv/_SUCCESS
csv/part-00000-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv
csv/part-00001-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv
csv/part-00002-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv


In [17]:
cat(bucket,"csv/part")

File: csv/part-00000-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv
----------------------
1,alex,2019-01-01,1000

######################
File: csv/part-00001-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv
----------------------
2,alex,2019-02-01,1500
4,maria,2020-01-01,5000

######################
File: csv/part-00002-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv
----------------------
3,alex,2019-03-01,1700

######################


In [18]:
read_csv=spark.read.format("csv").load(f"s3://{bucket}/csv")

read_csv.printSchema()
read_csv.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



+---+-----+----------+----+
|_c0|  _c1|       _c2| _c3|
+---+-----+----------+----+
|  2| alex|2019-02-01|1500|
|  4|maria|2020-01-01|5000|
|  1| alex|2019-01-01|1000|
|  3| alex|2019-03-01|1700|
+---+-----+----------+----+



In [19]:
write_csv=(df3
           .write
           .format("csv")
           .mode("append") # append
           .save(f"s3://{bucket}/csv")
          )

In [20]:
ls(bucket,"csv")

csv/_SUCCESS
csv/part-00000-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv
csv/part-00000-9fd43bb8-0c13-4e00-8324-babfab453bd2-c000.csv
csv/part-00001-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv
csv/part-00001-9fd43bb8-0c13-4e00-8324-babfab453bd2-c000.csv
csv/part-00002-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv


In [21]:
cat(bucket,"csv/part")

File: csv/part-00000-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv
----------------------
1,alex,2019-01-01,1000

######################
File: csv/part-00000-9fd43bb8-0c13-4e00-8324-babfab453bd2-c000.csv
----------------------

######################
File: csv/part-00001-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv
----------------------
2,alex,2019-02-01,1500
4,maria,2020-01-01,5000

######################
File: csv/part-00001-9fd43bb8-0c13-4e00-8324-babfab453bd2-c000.csv
----------------------
1,otto,2019-10-01,4444,neue Spalte 1

######################
File: csv/part-00002-20873225-f284-4d1d-88b7-5762e628b2c4-c000.csv
----------------------
3,alex,2019-03-01,1700

######################


In [54]:
read_csv=spark.read.format("csv").load(f"s3://{bucket}/csv")

read_csv.printSchema()
read_csv.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)

+---+-----+----------+----+
|_c0|  _c1|       _c2| _c3|
+---+-----+----------+----+
|  2| alex|2019-02-01|1500|
|  4|maria|2020-01-01|5000|
|  1| otto|2019-10-01|4444|
|  1| alex|2019-01-01|1000|
|  3| alex|2019-03-01|1700|
+---+-----+----------+----+



* kein Schema (Typen)
* kein anfügen neuer Spalten

## JSON

In [23]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_json=(df1
           .write
           .format("json")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/json")
          )


Number of Partitions: 3


In [24]:
ls(bucket,"json")

json/_SUCCESS
json/part-00000-5b8b02be-f698-4cf6-9320-3aa2642f2997-c000.json
json/part-00001-5b8b02be-f698-4cf6-9320-3aa2642f2997-c000.json
json/part-00002-5b8b02be-f698-4cf6-9320-3aa2642f2997-c000.json


In [25]:
cat(bucket,"json/part")

File: json/part-00000-5b8b02be-f698-4cf6-9320-3aa2642f2997-c000.json
----------------------
{"id":1,"account":"alex","dt_transaction":"2019-01-01","balance":1000}

######################
File: json/part-00001-5b8b02be-f698-4cf6-9320-3aa2642f2997-c000.json
----------------------
{"id":2,"account":"alex","dt_transaction":"2019-02-01","balance":1500}
{"id":4,"account":"maria","dt_transaction":"2020-01-01","balance":5000}

######################
File: json/part-00002-5b8b02be-f698-4cf6-9320-3aa2642f2997-c000.json
----------------------
{"id":3,"account":"alex","dt_transaction":"2019-03-01","balance":1700}

######################


In [26]:
write_csv=(df3
           .write
           .format("json")
           .mode("append") # append
           .save(f"s3://{bucket}/json")
          )

In [27]:
read_json=spark.read.format("json").load(f"s3://{bucket}/json")

read_json.printSchema()
read_json.show()

root
 |-- account: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- dt_transaction: string (nullable = true)
 |-- id: long (nullable = true)
 |-- new: string (nullable = true)

+-------+-------+--------------+---+-------------+
|account|balance|dt_transaction| id|          new|
+-------+-------+--------------+---+-------------+
|   alex|   1500|    2019-02-01|  2|         null|
|  maria|   5000|    2020-01-01|  4|         null|
|   otto|   4444|    2019-10-01|  1|neue Spalte 1|
|   alex|   1000|    2019-01-01|  1|         null|
|   alex|   1700|    2019-03-01|  3|         null|
+-------+-------+--------------+---+-------------+



* Kein Schema
* Neue Spalten werden als neues Attribut hinzugefügt

## Avro

In [28]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_avro=(df1
           .write
           .format("avro")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/avro")
          )

Number of Partitions: 3


In [29]:
ls(bucket,"avro")

avro/_SUCCESS
avro/part-00000-66653003-5264-4bf8-b5c9-64ab10b3c708-c000.avro
avro/part-00001-66653003-5264-4bf8-b5c9-64ab10b3c708-c000.avro
avro/part-00002-66653003-5264-4bf8-b5c9-64ab10b3c708-c000.avro


In [30]:
cat(bucket,"avro/part",True)

File: avro/part-00000-66653003-5264-4bf8-b5c9-64ab10b3c708-c000.avro
----------------------
b'Obj\x01\x06\x16avro.schema\xfa\x03{"type":"record","name":"topLevelRecord","fields":[{"name":"id","type":["long","null"]},{"name":"account","type":["string","null"]},{"name":"dt_transaction","type":[{"type":"int","logicalType":"date"},"null"]},{"name":"balance","type":["long","null"]}]}0org.apache.spark.version\n3.3.1\x14avro.codec\x0csnappy\x00\x0b\xb8\xac7\xef\x1c}i|\xf7\x7fi\x0eA\xbe\\\x02*\x0f8\x00\x02\x00\x08alex\x00\xd2\x97\x02\x00\xd0\x0f\x84/\x04]\x0b\xb8\xac7\xef\x1c}i|\xf7\x7fi\x0eA\xbe\\'
######################
File: avro/part-00001-66653003-5264-4bf8-b5c9-64ab10b3c708-c000.avro
----------------------
b'Obj\x01\x06\x16avro.schema\xfa\x03{"type":"record","name":"topLevelRecord","fields":[{"name":"id","type":["long","null"]},{"name":"account","type":["string","null"]},{"name":"dt_transaction","type":[{"type":"int","logicalType":"date"},"null"]},{"name":"balance","type":["long","null"]

In [31]:
read_json=spark.read.format("avro").load(f"s3://{bucket}/avro")
read_json.printSchema()
read_json.show()

root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)



+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  2|   alex|    2019-02-01|   1500|
|  4|  maria|    2020-01-01|   5000|
|  1|   alex|    2019-01-01|   1000|
|  3|   alex|    2019-03-01|   1700|
+---+-------+--------------+-------+



In [32]:
write_avro=(df3
           .write
           .format("avro")
           .mode("append") # append
           .save(f"s3://{bucket}/avro")
          )

In [33]:
read_json=spark.read.format("avro").load(f"s3://{bucket}/avro")
read_json.printSchema()
read_json.show()

root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)
 |-- new: string (nullable = true)



+---+-------+--------------+-------+-------------+
| id|account|dt_transaction|balance|          new|
+---+-------+--------------+-------+-------------+
|  1|   otto|    2019-10-01|   4444|neue Spalte 1|
|  2|   alex|    2019-02-01|   1500|         null|
|  4|  maria|    2020-01-01|   5000|         null|
|  1|   alex|    2019-01-01|   1000|         null|
|  3|   alex|    2019-03-01|   1700|         null|
+---+-------+--------------+-------+-------------+



* Schema erhalten
* Schema evolution 

## Parquet

In [34]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_parquet=(df1
           .write
           .partitionBy("account")
           .format("parquet")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/parquet")
          )


Number of Partitions: 3


In [35]:
ls(bucket,"parquet")

parquet/_SUCCESS
parquet/account=alex/part-00000-2ffc4cd5-064e-491f-bd92-117792787ed3.c000.snappy.parquet
parquet/account=alex/part-00001-2ffc4cd5-064e-491f-bd92-117792787ed3.c000.snappy.parquet
parquet/account=alex/part-00002-2ffc4cd5-064e-491f-bd92-117792787ed3.c000.snappy.parquet
parquet/account=maria/part-00001-2ffc4cd5-064e-491f-bd92-117792787ed3.c000.snappy.parquet


In [36]:
cat(bucket,"parquet/account=maria",True)

File: parquet/account=maria/part-00001-2ffc4cd5-064e-491f-bd92-117792787ed3.c000.snappy.parquet
----------------------
b'PAR1\x15\x00\x15\x1c\x15 \x15\xb6\xf1\xd0\xb8\x02\x1c\x15\x02\x15\x00\x15\x06\x15\x08\x00\x00\x0e4\x02\x00\x00\x00\x03\x01\x04\x00\x00\x00\x00\x00\x00\x00\x15\x00\x15\x14\x15\x18\x15\xf9\xb6\xd3\xce\x0f\x1c\x15\x02\x15\x00\x15\x06\x15\x08\x00\x00\n$\x02\x00\x00\x00\x03\x01VG\x00\x00\x15\x00\x15\x1c\x15 \x15\xba\xaa\xc0\x92\x08\x1c\x15\x02\x15\x00\x15\x06\x15\x08\x00\x00\x0e4\x02\x00\x00\x00\x03\x01\x88\x13\x00\x00\x00\x00\x00\x00\x19\x11\x02\x19\x18\x08\x04\x00\x00\x00\x00\x00\x00\x00\x19\x18\x08\x04\x00\x00\x00\x00\x00\x00\x00\x15\x02\x19\x16\x00\x00\x19\x11\x02\x19\x18\x04VG\x00\x00\x19\x18\x04VG\x00\x00\x15\x02\x19\x16\x00\x00\x19\x11\x02\x19\x18\x08\x88\x13\x00\x00\x00\x00\x00\x00\x19\x18\x08\x88\x13\x00\x00\x00\x00\x00\x00\x15\x02\x19\x16\x00\x00\x19\x1c\x16\x08\x15N\x16\x00\x00\x00\x19\x1c\x16V\x15F\x16\x00\x00\x00\x19\x1c\x16\x9c\x01\x15N\x16\x00\x00\x00\x15\x

In [37]:
read_parquet=(spark
              .read.format("parquet")
              .load(f"s3://{bucket}/parquet")
              .filter(col("account")=="alex")
             )

read_parquet.printSchema()
read_parquet.show()

root
 |-- id: long (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)
 |-- account: string (nullable = true)



+---+--------------+-------+-------+
| id|dt_transaction|balance|account|
+---+--------------+-------+-------+
|  1|    2019-01-01|   1000|   alex|
|  2|    2019-02-01|   1500|   alex|
|  3|    2019-03-01|   1700|   alex|
+---+--------------+-------+-------+



## Delta

- a **storage layer** that runs on top of existing data lakes
- supports ACID transactions and data versioning
- allows data lineage tracking
- provides optimization for streaming workloads

In [38]:
write_delta=(df1
           .write
           .format("delta")
           .option("mergeSchema", "false")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/delta")
          )

In [39]:
ls(bucket,"delta")

delta/_delta_log/00000000000000000000.json
delta/part-00000-71e8256a-4ae9-4a30-82df-1df4300074fa-c000.snappy.parquet
delta/part-00001-740db38d-df85-4d5f-b675-785b8e5a1ff6-c000.snappy.parquet
delta/part-00002-751ee4f7-7ea7-4731-9df7-9a881d3a97ac-c000.snappy.parquet


In [40]:
ls(bucket,"delta/_delta_log")

delta/_delta_log/00000000000000000000.json


In [41]:
cat(bucket,"delta/_delta_log")

File: delta/_delta_log/00000000000000000000.json
----------------------
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"cc178cae-6af7-410b-9996-f81ccdeb1ab5","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"account\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"dt_transaction\",\"type\":\"date\",\"nullable\":true,\"metadata\":{}},{\"name\":\"balance\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1684095467225}}
{"add":{"path":"part-00000-71e8256a-4ae9-4a30-82df-1df4300074fa-c000.snappy.parquet","partitionValues":{},"size":1236,"modificationTime":1684095469000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":1,\"account\":\"alex\",\"dt_transaction\":\"2019-01-01\",\"balance\":1000},\"maxValues\":{\"id\":1,\"account\":\"alex\",\"dt

In [42]:
write_delta=(df2
           .write
           .format("delta")
           .mode("append") # append
           .save(f"s3://{bucket}/delta")
          )

In [43]:
write_delta=(df3
           .write
           .format("delta")
           .option("mergeSchema", "true")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/delta")
          )

In [44]:
cat(bucket,"delta/_delta_log")

File: delta/_delta_log/00000000000000000000.json
----------------------
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"cc178cae-6af7-410b-9996-f81ccdeb1ab5","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"account\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"dt_transaction\",\"type\":\"date\",\"nullable\":true,\"metadata\":{}},{\"name\":\"balance\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1684095467225}}
{"add":{"path":"part-00000-71e8256a-4ae9-4a30-82df-1df4300074fa-c000.snappy.parquet","partitionValues":{},"size":1236,"modificationTime":1684095469000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":1,\"account\":\"alex\",\"dt_transaction\":\"2019-01-01\",\"balance\":1000},\"maxValues\":{\"id\":1,\"account\":\"alex\",\"dt

In [58]:
DeltaTable

delta.tables.DeltaTable

In [59]:
deltaTable = DeltaTable.forPath(spark, f"s3://{bucket}/delta")
# --> Vermutlich falsche Delta Version zu Spark
#fullHistoryDF = deltaTable.history()    # get the full history of the table

Py4JError: An error occurred while calling z:io.delta.tables.DeltaTable.forPath. Trace:
py4j.Py4JException: Method forPath([class org.apache.spark.sql.SparkSession, class java.lang.String, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Unknown Source)



In [ ]:
fullHistoryDF.select("version","readVersion","timestamp","userId","operation","operationParameters","operationMetrics","userMetadata").show(truncate=True)

In [60]:
spark.read.format("delta").load(f"s3://{bucket}/delta").show()

+---+-------+--------------+-------+-------------+
| id|account|dt_transaction|balance|          new|
+---+-------+--------------+-------+-------------+
|  1|   otto|    2019-10-01|   4444|neue Spalte 1|
+---+-------+--------------+-------+-------------+



## Delta: Time travel

In [47]:
spark.read.format("delta").option("versionAsOf", "1").load(f"s3://{bucket}/delta").show()


+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  2|   alex|    2019-02-01|   1500|
|  4|  maria|    2020-01-01|   5000|
|  2|  peter|    2021-01-01|    100|
|  3|   alex|    2019-03-01|   1700|
|  1|   alex|    2019-03-01|   3300|
|  1|   alex|    2019-01-01|   1000|
+---+-------+--------------+-------+



## Delta: Merge

In [50]:
deltaTable2 = DeltaTable.forPath(spark, f"s3://{bucket}/delta")


df2a=df2.withColumn("new",lit("test"))
df2a.show()
deltaTable2.toDF().show()

Py4JError: An error occurred while calling z:io.delta.tables.DeltaTable.forPath. Trace:
py4j.Py4JException: Method forPath([class org.apache.spark.sql.SparkSession, class java.lang.String, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Unknown Source)



In [49]:
dt3=(deltaTable2.alias("oldData")
      .merge(df2a.alias("newData"),
            "oldData.account = newData.account AND oldData.dt_transaction = newData.dt_transaction")
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
      .execute()
    )

deltaTable2.toDF().show()

NameError: name 'deltaTable2' is not defined

In [ ]:
result=(deltaTable2
        .toDF()
        .withColumn("month",month(col("dt_transaction")))
        .groupBy("account","month").agg(sum("balance"))
        .sort("account","month")
       )
result.show()

In [ ]:
result=(spark.read
        .format("delta")
        .option("versionAsOf", "1")
        .load("output/delta")
        .withColumn("month",month(col("dt_transaction")))
        .groupBy("account","month").agg(sum("balance"))
        .sort("account","month")
       )
result.show()

* Schema
* Schema evolution
* Transaction Log
* Time Travel

## Iceberg

- a **table format**
- supports schema evolution and provides a portable table metadata format
- best suited for analytical workloads

In [51]:
print("Current Catalog:",spark.catalog.currentDatabase())
print("List Catalogs:",spark.catalog.listDatabases())
print("List Tables in current Catalog:",spark.catalog.listTables())

Current Catalog: default
List Catalogs: [Database(name='default', description='Default Hive database', locationUri='file:/home/hive/warehouse')]
List Tables in current Catalog: []


In [61]:
# create a Database(name=<db_name>, locationUri='s3a://<bucket>/')
spark.sql(f"CREATE DATABASE iceberg_db LOCATION 's3a://{bucket}/'")

DataFrame[]

In [62]:
### show databases and tables in iceberg catalog (only sees iceberg formated tables)
# all databases from hive are shown
spark.sql("SHOW databases from ice").show()

+----------+
| namespace|
+----------+
|   default|
|iceberg_db|
+----------+



In [63]:
spark.sql("show tables from iceberg_db").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [ ]:
#### Delete Iceberg tables: first drop the table 
#spark.sql("drop table iceberg_db.iceberg_table")
#delete_objects("aleks-test", "iceberg_table")

In [64]:
write_iceberg=(df1
                  .write
                  .format("iceberg")
                  .mode("overwrite")
                  .saveAsTable("iceberg_db.iceberg")
               )


In [66]:
ls(bucket,"iceberg")

iceberg/data/00000-545-3b1bf31a-0ecb-4245-8a30-5d6b4de18962-00001.parquet
iceberg/data/00000-550-7fa8b7a8-5984-487f-b524-9753864c2808-00001.parquet
iceberg/data/00001-546-96c2733b-2dc9-4e04-b06d-1c2cbf6fe159-00001.parquet
iceberg/data/00001-551-6cfea253-4caa-4fb6-bc7e-e3d9f956388b-00001.parquet
iceberg/data/00002-547-3a8580fe-e207-4cf4-8015-2e6f91d99deb-00001.parquet
iceberg/metadata/00000-2346cea3-3db3-46a4-bb55-419ae993156b.metadata.json
iceberg/metadata/00001-70cc0e6d-feed-4202-bf2a-587f8dd81bbe.metadata.json
iceberg/metadata/5dc0a4c4-3c72-4d16-a200-c299b635415c-m0.avro
iceberg/metadata/73a18816-64a1-4119-8453-bb671b8abfb3-m0.avro
iceberg/metadata/snap-2282180466624073266-1-73a18816-64a1-4119-8453-bb671b8abfb3.avro
iceberg/metadata/snap-4263160168885610306-1-5dc0a4c4-3c72-4d16-a200-c299b635415c.avro


In [65]:

write_iceberg=(df2
                   .write
                   .format("iceberg")
                   .mode("append") # append
                   .saveAsTable("iceberg_db.iceberg")
                 )

In [67]:
ls(bucket,"iceberg")

iceberg/data/00000-545-3b1bf31a-0ecb-4245-8a30-5d6b4de18962-00001.parquet
iceberg/data/00000-550-7fa8b7a8-5984-487f-b524-9753864c2808-00001.parquet
iceberg/data/00001-546-96c2733b-2dc9-4e04-b06d-1c2cbf6fe159-00001.parquet
iceberg/data/00001-551-6cfea253-4caa-4fb6-bc7e-e3d9f956388b-00001.parquet
iceberg/data/00002-547-3a8580fe-e207-4cf4-8015-2e6f91d99deb-00001.parquet
iceberg/metadata/00000-2346cea3-3db3-46a4-bb55-419ae993156b.metadata.json
iceberg/metadata/00001-70cc0e6d-feed-4202-bf2a-587f8dd81bbe.metadata.json
iceberg/metadata/5dc0a4c4-3c72-4d16-a200-c299b635415c-m0.avro
iceberg/metadata/73a18816-64a1-4119-8453-bb671b8abfb3-m0.avro
iceberg/metadata/snap-2282180466624073266-1-73a18816-64a1-4119-8453-bb671b8abfb3.avro
iceberg/metadata/snap-4263160168885610306-1-5dc0a4c4-3c72-4d16-a200-c299b635415c.avro


In [69]:
cat(bucket,"iceberg/metadata/00000-2346cea3-3db3-46a4-bb55-419ae993156b.metadata.json",False)

File: iceberg/metadata/00000-2346cea3-3db3-46a4-bb55-419ae993156b.metadata.json
----------------------
{
  "format-version" : 1,
  "table-uuid" : "c1a84965-5daa-434b-8f3a-3f27cd91ac55",
  "location" : "s3a://fileformats//iceberg",
  "last-updated-ms" : 1684152769085,
  "last-column-id" : 4,
  "schema" : {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "id",
      "required" : false,
      "type" : "long"
    }, {
      "id" : 2,
      "name" : "account",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 3,
      "name" : "dt_transaction",
      "required" : false,
      "type" : "date"
    }, {
      "id" : 4,
      "name" : "balance",
      "required" : false,
      "type" : "long"
    } ]
  },
  "current-schema-id" : 0,
  "schemas" : [ {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "id",
      "required" : false,
      "type" : "long"
    }, {
      "id" : 2,
      "n

In [70]:
#ALTER TABLE myTable ADD COLUMNS (address VARCHAR) - the number of columns in the df3 does not match the schema of the table, so we modify the schema of the existing table
spark.sql("ALTER TABLE iceberg_db.iceberg ADD COLUMNS (new VARCHAR(50))")

DataFrame[]

In [71]:
write_iceberg=(df3
                  .write
                  .format("iceberg")
                  .mode("append") # append
                  .option("schema", schema3)
                  .saveAsTable("iceberg_db.iceberg")
                  )

In [72]:
## Read Iceberg table:

iceberg_df = spark.read.table("iceberg_db.iceberg")
iceberg_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)
 |-- new: string (nullable = true)



In [73]:

spark.sql("SELECT * FROM iceberg_db.iceberg.history;").show()
spark.sql("SELECT * FROM iceberg_db.iceberg.files;").show()
spark.sql("SELECT * FROM iceberg_db.iceberg.snapshots;").show()

## alternative syntax example:
# spark.read.format("iceberg").load("iceberg_db.iceberg_table.files").show()


+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2023-05-15 14:12:...|2282180466624073266|               null|               true|
|2023-05-15 14:13:...|4263160168885610306|2282180466624073266|               true|
|2023-05-15 14:16:...|3806458280092384569|4263160168885610306|               true|
+--------------------+-------------------+-------------------+-------------------+



+-------+--------------------+-----------+-------+------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------+-------------+------------+-------------+
|content|           file_path|file_format|spec_id|record_count|file_size_in_bytes|        column_sizes|        value_counts|   null_value_counts|nan_value_counts|        lower_bounds|        upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|
+-------+--------------------+-----------+-------+------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------+-------------+------------+-------------+
|      0|s3a://fileformats...|    PARQUET|      0|           1|              1534|{1 -> 52, 2 -> 55...|{1 -> 1, 2 -> 1, ...|{1 -> 0, 2 -> 0, ...|              {}|{1 ->        , 2...|{1 ->        , 2...|        null|      

### Iceberg: Time Travel
- ```snapshot-id``` selects a specific table snapshot
- ```as-of-timestamp``` selects the current snapshot at a timestamp, in milliseconds
- ```branch``` selects the head snapshot of the specified branch. Note that currently branch cannot be combined with as-of-timestamp.
- ```tag``` selects the snapshot associated with the specified tag. Tags cannot be combined with as-of-timestamp.

In [74]:
# from the results of iceberg_table.snapshots get the snapshots IDs
snapshot1 = spark.read \
                 .option("snapshot-id", "2282180466624073266") \
                 .format("iceberg") \
                 .load("iceberg_db.iceberg").show()

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  1|   alex|    2019-01-01|   1000|
|  2|   alex|    2019-02-01|   1500|
|  4|  maria|    2020-01-01|   5000|
|  3|   alex|    2019-03-01|   1700|
+---+-------+--------------+-------+



In [76]:
snapshot2 = spark.read \
                 .option("snapshot-id", "4263160168885610306") \
                 .format("iceberg") \
                 .load("iceberg_db.iceberg").show()

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  1|   alex|    2019-01-01|   1000|
|  2|   alex|    2019-02-01|   1500|
|  4|  maria|    2020-01-01|   5000|
|  3|   alex|    2019-03-01|   1700|
|  1|   alex|    2019-03-01|   3300|
|  2|  peter|    2021-01-01|    100|
+---+-------+--------------+-------+



In [81]:
tsToExpire = f.current_timestamp() - timedelta(minutes=10)
print(tsToExpire)

Column<'(current_timestamp() - INTERVAL '0 00:10:00' DAY TO SECOND)'>


In [82]:
## need iceberg.table
## geth nicht verstehe ich. noch nicht??
table.expireSnapshots().expireOlderThan(tsToExpire).commit();

NameError: name 'table' is not defined

# Hudi

- a **storage abstraction layer** 
- enables data ingestion and query capability on large-scale, evolving datasets
- well-suited for real-time streaming workloads and batch processing

In [83]:
# update partition path, i.e. "id/dt_transaction"
record_key = "id"
partition_path = "id"

hudi_options = {
    "hoodie.table.name": df1,
    "hoodie.datasource.write.recordkey.field": record_key,
    "hoodie.datasource.write.partitionpath.field": partition_path,
    "hoodie.datasource.write.table.name": df1,
    "hoodie.datasource.write.operation": "upsert",
    "hoodie.datasource.write.precombine.field": "ts",  # This field is used by Hoodie to resolve conflicts between records with the same key (in this case, id) 
    "hoodie.upsert.shuffle.parallelism": 2,
    "hoodie.insert.shuffle.parallelism": 2
}

In [84]:
write_hudi=(df1.withColumn("ts", f.current_timestamp()).write.format("hudi") # "ts" field is a mandatory field in Hoodie that specifies the timestamp of the record, so we add a new column and use simple current_timestamp() function
               .options(**hudi_options)
               .mode("overwrite")
               .save(f"s3://{bucket}/hudi")
               )

In [85]:
ls(bucket,"hudi")

hudi/.hoodie/.aux/.bootstrap/.fileids/
hudi/.hoodie/.aux/.bootstrap/.partitions/
hudi/.hoodie/.schema/
hudi/.hoodie/.temp/
hudi/.hoodie/20230515122157032.commit
hudi/.hoodie/20230515122157032.commit.requested
hudi/.hoodie/20230515122157032.inflight
hudi/.hoodie/archived/
hudi/.hoodie/hoodie.properties
hudi/.hoodie/metadata/.hoodie/.aux/.bootstrap/.fileids/
hudi/.hoodie/metadata/.hoodie/.aux/.bootstrap/.partitions/
hudi/.hoodie/metadata/.hoodie/.schema/
hudi/.hoodie/metadata/.hoodie/.temp/
hudi/.hoodie/metadata/.hoodie/00000000000000.deltacommit
hudi/.hoodie/metadata/.hoodie/00000000000000.deltacommit.inflight
hudi/.hoodie/metadata/.hoodie/00000000000000.deltacommit.requested
hudi/.hoodie/metadata/.hoodie/20230515122157032.deltacommit
hudi/.hoodie/metadata/.hoodie/20230515122157032.deltacommit.inflight
hudi/.hoodie/metadata/.hoodie/20230515122157032.deltacommit.requested
hudi/.hoodie/metadata/.hoodie/archived/
hudi/.hoodie/metadata/.hoodie/hoodie.properties
hudi/.hoodie/metadata/files/.

In [86]:
write_hudi=(df2.withColumn("ts", f.current_timestamp()).write.format("hudi") # "ts" field is a mandatory field in Hoodie that specifies the timestamp of the record, so we add a new column and use simple current_timestamp() function
               .options(**hudi_options)
               .mode("append")
               .save(f"s3://{bucket}/hudi")
               )

# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf


# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]


In [87]:
write_hudi=(df3.withColumn("ts", f.current_timestamp()).write.format("hudi") # "ts" field is a mandatory field in Hoodie that specifies the timestamp of the record, so we add a new column and use simple current_timestamp() function
               .options(**hudi_options)
               .mode("append")
               .save(f"s3://{bucket}/hudi")
               )

In [88]:
df_hudi = spark.read.format("hudi").load(f"s3://{bucket}/hudi").show()

+-------------------+--------------------+------------------+----------------------+--------------------+-------+--------------+-------+-------------+--------------------+---+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|account|dt_transaction|balance|          new|                  ts| id|
+-------------------+--------------------+------------------+----------------------+--------------------+-------+--------------+-------+-------------+--------------------+---+
|  20230515122339203|20230515122339203...|                 1|                     1|cf0f8517-23b9-441...|   otto|    2019-10-01|   4444|neue Spalte 1|2023-05-15 14:23:...|  1|
|  20230515122157032|20230515122157032...|                 4|                     4|ee5e9af9-3f2d-471...|  maria|    2020-01-01|   5000|         null|2023-05-15 14:21:...|  4|
|  20230515122321620|20230515122321620...|                 2|                     2|0cd93c57-fb35-4ff...|  peter|    202

In [89]:
ls(bucket,"hudi")

hudi/.hoodie/.aux/.bootstrap/.fileids/
hudi/.hoodie/.aux/.bootstrap/.partitions/
hudi/.hoodie/.schema/
hudi/.hoodie/.temp/
hudi/.hoodie/20230515122157032.commit
hudi/.hoodie/20230515122157032.commit.requested
hudi/.hoodie/20230515122157032.inflight
hudi/.hoodie/20230515122321620.commit
hudi/.hoodie/20230515122321620.commit.requested
hudi/.hoodie/20230515122321620.inflight
hudi/.hoodie/20230515122339203.commit
hudi/.hoodie/20230515122339203.commit.requested
hudi/.hoodie/20230515122339203.inflight
hudi/.hoodie/archived/
hudi/.hoodie/hoodie.properties
hudi/.hoodie/metadata/.hoodie/.aux/.bootstrap/.fileids/
hudi/.hoodie/metadata/.hoodie/.aux/.bootstrap/.partitions/
hudi/.hoodie/metadata/.hoodie/.schema/
hudi/.hoodie/metadata/.hoodie/.temp/
hudi/.hoodie/metadata/.hoodie/00000000000000.deltacommit
hudi/.hoodie/metadata/.hoodie/00000000000000.deltacommit.inflight
hudi/.hoodie/metadata/.hoodie/00000000000000.deltacommit.requested
hudi/.hoodie/metadata/.hoodie/20230515122157032.deltacommit
hudi

#### Hudi: Time Travel 

In [90]:
## Get the commit time from the Hudi table

spark.read.format("hudi")\
     .option("as.of.instant", "20230515122339203")\
     .load(f"s3://{bucket}/hudi").show()

+-------------------+--------------------+------------------+----------------------+--------------------+-------+--------------+-------+-------------+--------------------+---+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|account|dt_transaction|balance|          new|                  ts| id|
+-------------------+--------------------+------------------+----------------------+--------------------+-------+--------------+-------+-------------+--------------------+---+
|  20230515122339203|20230515122339203...|                 1|                     1|cf0f8517-23b9-441...|   otto|    2019-10-01|   4444|neue Spalte 1|2023-05-15 14:23:...|  1|
|  20230515122157032|20230515122157032...|                 4|                     4|ee5e9af9-3f2d-471...|  maria|    2020-01-01|   5000|         null|2023-05-15 14:21:...|  4|
|  20230515122321620|20230515122321620...|                 2|                     2|0cd93c57-fb35-4ff...|  peter|    202

In [91]:
account_data4 = [
    (5,"anna","2020-11-01",2000,"neue Spalte 1")
]
df4 = spark.createDataFrame(data=account_data4, schema = schema3).withColumn("dt_transaction",col("dt_transaction").cast("date")).repartition(3)

write_hudi=(df4.withColumn("ts", f.current_timestamp()).write.format("hudi")
               .options(**hudi_options)
               .mode("append")
               .save(f"s3://{bucket}/hudi")
               )

In [92]:
## Incremental query:

spark.read.format("hudi"). \
  load(f"s3://{bucket}/hudi"). \
  createOrReplaceTempView("hudi_snapshots")

In [93]:
commits = list(map(lambda row: row[0], spark.sql("select distinct(_hoodie_commit_time) as commitTime from hudi_snapshots order by commitTime").limit(10).collect()))
print(commits)

beginTime = commits[len(commits) - 4] # commit time we are interested in


['20230515122157032', '20230515122321620', '20230515122339203', '20230515123027657']


In [95]:
# incrementally query data
incremental_read_options = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': beginTime,
}

hudiIncrementalDF = spark.read.format("hudi"). \
  options(**incremental_read_options). \
  load(f"s3://{bucket}/hudi")
hudiIncrementalDF .createOrReplaceTempView("hudi_incremental")

spark.sql("select `_hoodie_commit_time`, account, balance, dt_transaction, ts from hudi_incremental").show(truncate=False)

+-------------------+-------+-------+--------------+--------------------------+
|_hoodie_commit_time|account|balance|dt_transaction|ts                        |
+-------------------+-------+-------+--------------+--------------------------+
|20230515122339203  |otto   |4444   |2019-10-01    |2023-05-15 14:23:39.428869|
|20230515123027657  |anna   |2000   |2020-11-01    |2023-05-15 14:30:27.805069|
|20230515122321620  |peter  |100    |2021-01-01    |2023-05-15 14:23:21.809413|
+-------------------+-------+-------+--------------+--------------------------+



### Hudi: Table maintenance
Hudi can run async or inline table services while running Strucrured Streaming query and takes care of cleaning, compaction and clustering. There's no operational overhead for the user.
For CoW tables, table services work in inline mode by default.
For MoR tables, some async services are enabled by default.

In [ ]:
spark.stop()